In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math

SAMPLES = 1000
SEED = 1337

np.random.seed(SEED)
tf.random.set_seed(SEED)
xValues = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)
np.random.shuffle(xValues)
yValues = np.sin(xValues)

plt.plot(xValues, yValues, 'b.', label="Train")
plt.legend()
plt.show()

In [ ]:
yValues += 0.1 * np.random.randn(*yValues.shape)

plt.plot(xValues, yValues, 'b.', label="Train")
plt.legend()
plt.show()

In [ ]:
TRAIN_SPLIT = int(0.6 * SAMPLES)
TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)

xTrain, xValidate, xTest = np.split(xValues, [TRAIN_SPLIT, TEST_SPLIT])
yTrain, yValidate, yTest = np.split(yValues, [TRAIN_SPLIT, TEST_SPLIT])

plt.plot(xTrain, yTrain, 'b.', label="Train")
plt.plot(xValidate, yValidate, 'y.', label="Validate")
plt.plot(xTest, yTest, 'r.', label="Test")
plt.legend()
plt.show()


In [ ]:
from keras import layers

model_1 = tf.keras.Sequential()
model_1.add(layers.Dense(16, activation='relu', input_shape=(1,)))
model_1.add(layers.Dense(1))
model_1.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model_1.summary()

history_1 = model_1.fit(xTrain, yTrain, epochs=500, batch_size=16, validation_data=(xValidate, yValidate))

In [ ]:

loss = history_1.history['loss']
valLoss = history_1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g.', label='Training loss')
plt.plot(epochs, valLoss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
predictions = model_1.predict(xTrain)

plt.clf()
plt.title('Training data predicted vs actual values')
plt.plot(xTest, yTest, 'b.', label='Actual')
plt.plot(xTrain, predictions, 'r.', label='Predicted')
plt.legend()
plt.show()

In [ ]:
model_2 = tf.keras.Sequential()
model_2.add(layers.Dense(16, activation='relu', input_shape=(1,)))
model_2.add(layers.Dense(16, activation='relu'))
model_2.add(layers.Dense(1))
model_2.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model_2.summary()
history_2 = model_2.fit(xTrain, yTrain, epochs=250, batch_size=16, validation_data=(xValidate, yValidate))


In [ ]:
loss = model_2.evaluate(xTest, yTest)
predictions = model_2.predict(xTest)
plt.clf()
plt.title('Comparison of predictions and actual values')
plt.plot(xTest, yTest, 'b.', label='Actual')
plt.plot(xTest, predictions, 'r.', label='Predicted')
plt.legend()
plt.show()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_2)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

converter = tf.lite.TFLiteConverter.from_keras_model(model_2)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_dataset_generator():
 for value in xTest:
    yield [np.array(value, dtype=np.float32, ndmin=2)]

converter.representative_dataset = representative_dataset_generator
tflite_model = converter.convert()

open("model_quantized.tflite", "wb").write(tflite_model)